# The Basics with Agents SDK

We recently launched the [Agents SDK](https://openai.github.io/openai-agents-python/) to make it easier to orchestrate agentic workflows. In this cookbook, we have'll some fun composing a Golf Agent that can help with creating personalized practice plans and also make score predictions for scores around you.

- [Response API](https://platform.openai.com/docs/api-reference/responses)
- [File Search API](https://platform.openai.com/docs/guides/tools-file-search)
- [Web Search API](https://platform.openai.com/docs/guides/tools-web-search?api-mode=responses)

## Agent Workflow

For our design we divide the roles and resoponsibilities of agents by functionality. This architecture will help not only with dividing roles and responaibilities but also debugging and optimizations later on. Below are the following Agents and their scope 

![Local image](./Tee%20Time%20Finder.png)

| Agent                          | Description                                                                                                                                                                                                                                             |
|--------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Triage Agent                   | Responsible for understanding the user's initial intent and handing off to our other agents.                                                                                                                                                            |
| Practice Planner Agent         | Responsible for constructing a personalized and detailed practice plan. This agent will have access to the FileSearch API to query for relevant user shot information.                                                                              |
| Score Predictor & Course Finder | Responsible for finding nearby golf courses based on the user's request, making a score prediction for each golf course returned, and recommending club distance lengths for that day. This agent has access to both the FileSearch API and the WebSearch API. |


## Data Preparation & Vector Store Population

Before we can fire user requests into our Golf Agent, we need to upload the user's historical shot data into a Vector Store. Here is an example of the original golf data from a user's driving range practice session. 

| Date            | Player | Club Name | Club Type | Club Speed | Attack Angle | Club Path | Club Face | Face to Path | Ball Speed | Smash Factor | Launch Angle | Launch Direction | Backspin  | Sidespin  | Spin Rate | Spin Rate Type | Spin Axis | Apex Height | Carry Distance | Carry Deviation Angle | Carry Deviation Distance | Total Distance | Total Deviation Angle | Total Deviation Distance |
|-----------------|--------|-----------|-----------|-------------|---------------|------------|------------|----------------|--------------|----------------|----------------|------------------|------------|------------|-------------|----------------|------------|--------------|----------------|------------------------|--------------------------|----------------|------------------------|--------------------------|
| 1/4/25 17:01:39 | Brian  |           | 7 Iron    | 74.02      | 2.68         | 5.99      | 10.18     | 4.19           | 100.39      | 1.36           | 20.67         | 9.11              | 3501.39   | -366.16   | 3520.48    | Measured      | 5.97       | 22.77        | 143.31         | 11.92                  | 29.59                    | 156.29         | 12.01                  | 32.52                    |
| 1/4/25 17:02:08 | Brian  |           | 7 Iron    | 75.85      | -3.13        | 3.87      | 5.76      | 1.89           | 100.17      | 1.32           | 18.06         | 5.22              | 3549.35   | 61.95     | 3549.89    | Measured      | -1.0       | 19.23        | 140.03         | 5.18                   | 12.63                    | 154.17         | 5.11                   | 13.74                    |
| 1/4/25 17:02:52 | Jason  |           | 7 Iron    | 75.74      | -4.96        | 7.18      | 5.95      | -1.23          | 87.15       | 1.15           | 12.74         | 5.79              | 2423.86   | 1189.54   | 2700.02    | Estimated     | -26.14     | 6.79         | 91.81          | 0.92                   | 1.47                     | 122.10         | -0.35                  | -0.75                    |
| 1/4/25 17:03:49 | Herald |           | 7 Iron    | 75.99      | 4.53         | 5.91      | 5.20      | -0.71          | 96.95       | 1.28           | 14.49         | 5.01              | 4544.12   | -699.41   | 4597.63    | Measured      | 8.75       | 13.98        | 124.95         | 8.29                   | 18.01                    | 138.91         | 8.55                   | 20.66                    |
| 1/4/25 17:04:58 | Jason  |           | 7 Iron    | 77.08      | 7.36         | 3.77      | 3.15      | -0.62          | 102.56      | 1.33           |               | 3.06              | 1918.64   | 0.00      | 1918.64    | Measured      | 0.00       |              |                 |                         |                          |                |                        |                          |
| 1/4/25 17:05:42 | Brian  |           | 7 Iron    | 34.29      | 0.00         | 0.00      | 0.00      | 0.00           | 41.41       | 1.21           | 17.26         | -2.19             | 3052.55   | 0.00      | 3052.55    | Estimated     | 0.00       | 2.00         | 23.61          | -2.21                  | -0.91                    | 29.05          | -2.21                  | -1.12                    |
| 1/4/25 17:08:56 | Jason  |           | 7 Iron    | 75.77      | -4.19        | 6.90      | 10.15     | 3.25           | 98.76       | 1.30           | 19.33         | 9.20              | 5117.86   | 573.16    | 5149.85    | Measured      | -6.39      | 21.80        | 133.79         | 7.32                   | 17.04                    | 143.50         | 7.09                   | 17.72                    |
| 1/4/25 17:09:17 | Brian  |           | 7 Iron    | 76.39      | 2.79         | 3.26      | 6.68      | 3.42           | 104.35      | 1.37           | 15.02         | 5.89              | 3443.57   | -523.87   | 3483.19    | Measured      | 8.65       | 16.47        | 143.91         | 9.10                   | 22.75                    | 159.72         | 9.34                   | 25.93                    |
| 1/4/25 17:09:48 | Jason  |           | 7 Iron    | 76.35      | 0.35         | 2.09      | 6.56      | 4.47           | 102.63      | 1.34           | 14.20         | 5.65              | 5386.30   | -1082.16  | 5493.93    | Measured      | 11.36      | 16.61        | 135.01         | 10.57                  | 24.77                    | 146.59         | 10.88                  | 27.67                    |
| 1/4/25 17:10:26 | Lauren |           | 7 Iron    | 77.78      | 4.00         | 8.51      | 9.90      | 1.39           | 99.70       | 1.28           | 17.16         | 9.20              | 4462.13   | -68.54    | 4462.66    | Measured      | 0.88       | 18.77        | 135.86         | 10.13                  | 23.90                    | 147.86         | 10.09                  | 25.91                    |


In the perfect world, if we were a serious golfer, we could have added to the Note and Tag fields to writedown specific observations from each of these shots while practicing. Therefore, we're going to enrich this data by using a LLM to provide detailed shot feedback for each shot.

## Step 0: Setup Your Environment

In [17]:
!pip install pandas openai tqdm dotenv openai-agents ipywidgets jupyter_contrib_nbextensions

     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/23.5 MB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━ 22.8/23.5 MB 117.6 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 106.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached lxml-5.3.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata (3.7 kB)
Using cached lxml-5.3.1-cp311-cp311-macosx_10_9_x86_64.whl (4.4 MB)
  Created wheel for jupyter_contrib_nbextensions: filename=jupyter_contrib_nbextensions-0.7.0-py2.py3-none-any.whl size=23428843 sha256=67ef906b821ca6f6c5c7cc1654c789bc23afc4308f58c56abb0e4f06a24a43cf
  Stored in directory: /Users/jhall/Library/Caches/pip/wheels/cd/25/fe/cb6f3e82f5b1921b0157ac9e32adb2e54806ec1befc446be21
  Created wheel for jupyter_cont

In [22]:
import os
import time
import random
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()
client = OpenAI(api_key=os.getenv('API_KEY'))
tqdm.pandas()  # Enable progress_apply for pandas operations

## Step 1: Data Preparation
In this step, we read the input CSV files which contains the shot data. Then we are going to enrich the data by using `4o-mini` to analyze and provide feedback on each golfer's shot. We take this extra step to add semantic information for our vector store to use querying for results later on. The enriched files will be output into new CSV files in the `/processed` folder.

In [1]:
SYSTEM_INSTRUCTIONS = """
You are a professional and very popular golf instructor with years of experience. You have a deep understanding of a variety of golf drills and exercises that can help golfers improve their game. You excel at synthesizing complex golf metrics from launch monitors and distilling them into comprehensible and actionable feedback and tips without being too technical. Convert it into lamens terms for high handicappers.
"""

def generate_feedback(row):
    prompt = f"""
    Review the following shot data and provide concise feedback:

    Date: {row.get('Date', 'N/A')}
    Club Type: {row.get('Club Type', 'N/A')}
    Club Speed: {row.get('Club Speed', 'N/A')} mph
    Attack Angle: {row.get('Attack Angle', 'N/A')} degrees
    Club Path: {row.get('Club Path', 'N/A')} degrees
    Club Face: {row.get('Club Face', 'N/A')} degrees
    Face To Path: {row.get('Face To Path', 'N/A')} degrees
    Ball Speed: {row.get('Ball Speed', 'N/A')} mph
    Smash Factor: {row.get('Smash Factor', 'N/A')}
    Launch Angle: {row.get('Launch Angle', 'N/A')} degrees
    Launch Direction: {row.get('Launch Direction', 'N/A')} degrees
    Backspin: {row.get('Backspin', 'N/A')} rpm
    Sidespin: {row.get('Sidespin', 'N/A')} rpm
    Spin Rate: {row.get('Spin Rate', 'N/A')} rpm
    Spin Rate Type: {row.get('Spin Rate Type', 'N/A')}
    Spin Axis: {row.get('Spin Axis', 'N/A')} degrees
    Apex Height: {row.get('Apex Height', 'N/A')} yards
    Carry Distance: {row.get('Carry Distance', 'N/A')} yards
    Carry Deviation Angle: {row.get('Carry Deviation Angle', 'N/A')} degrees
    Carry Deviation Distance: {row.get('Carry Deviation Distance', 'N/A')} yards
    Total Distance: {row.get('Total Distance', 'N/A')} yards
    Total Deviation Angle: {row.get('Total Deviation Angle', 'N/A')} degrees
    Total Deviation Distance: {row.get('Total Deviation Distance', 'N/A')} yards
    Note: {row.get('Note', 'N/A')}
    Tag: {row.get('Tag', 'N/A')}
    Air Density: {row.get('Air Density', 'N/A')} g/L
    Temperature: {row.get('Temperature', 'N/A')} °F
    Air Pressure: {row.get('Air Pressure', 'N/A')} kPa
    Relative Humidity: {row.get('Relative Humidity', 'N/A')}%

    Provide brief feedback and a short tag describing a drill that the golfer can do to improve their game such as "step through drill", "swing path drill", "slow motion swing", "shoulder turn check", "half swing pause", etc.
    Format your response as JSON:
    {{
        "feedback": "Your feedback here",
        "tag": "Your tag here"
    }}
    """
    response = client.responses.create(
        model="gpt-4o-mini",
        input=[
            {"role": "system", "content": SYSTEM_INSTRUCTIONS},
            {"role": "user", "content": prompt}
        ],
        text={
            "format": {
                "type": "json_object"
            }
        }
    )

    result = response.output_text
    return pd.Series(eval(result))

def process_and_enrich_csv(filepath, output_filepath):
    df = pd.read_csv(filepath)
    feedback_tags = df.progress_apply(generate_feedback, axis=1)
    df['Note'] = feedback_tags['feedback']
    df['Tag'] = feedback_tags['tag']
    df.to_csv(output_filepath, index=False)
    print(f"Updated CSV saved to {output_filepath}")

data_folder = os.path.join(os.path.dirname(os.getcwd()), "basics_of_agents_sdk/data")
raw_data_folder = os.path.join(data_folder, 'raw')
enriched_data_folder = os.path.join(data_folder, 'processed')

# Loop through each CSV file in the raw data folder with a progress bar
for filename in tqdm(os.listdir(raw_data_folder), desc="Processing CSV files"):
    if filename.endswith('.csv'):
        csv_path = os.path.join(raw_data_folder, filename)
        output_filepath = os.path.join(enriched_data_folder, filename)
        process_and_enrich_csv(csv_path, output_filepath)


Processing CSV files:   8%|▊         | 1/13 [03:36<43:14, 216.19s/it]

Updated CSV saved to /Users/jhall/code/openai-cookbook/examples/agents_sdk/basics_of_agents_sdk/data/processed/Golf_Shot_Data_9.csv


Processing CSV files:  15%|█▌        | 2/13 [08:15<46:25, 253.22s/it]

Updated CSV saved to /Users/jhall/code/openai-cookbook/examples/agents_sdk/basics_of_agents_sdk/data/processed/Golf_Shot_Data_8.csv


Processing CSV files:  23%|██▎       | 3/13 [14:01<49:16, 295.65s/it]

...

Updated CSV saved to /Users/jhall/code/openai-cookbook/examples/agents_sdk/basics_of_agents_sdk/data/processed/Golf_Shot_Data_7.csv


Processing CSV files:  92%|█████████▏| 12/13 [37:46<03:06, 186.37s/it]

Updated CSV saved to /Users/jhall/code/openai-cookbook/examples/agents_sdk/basics_of_agents_sdk/data/processed/Golf_Shot_Data_5.csv


Processing CSV files: 100%|██████████| 13/13 [40:04<00:00, 184.99s/it]

Updated CSV saved to /Users/jhall/code/openai-cookbook/examples/agents_sdk/basics_of_agents_sdk/data/processed/Golf_Shot_Data_4.csv


## Step 3: Uploading to File Search API (Vector Store)
Now that we have our enriched data, we're going to chunk each CSV file by shot (i.e each row) and upload these files to our vector store.

In [ ]:
def upload_chunks_from_file(file_name, data_folder, purpose, max_retries=3, backoff_factor=2, use_cache=False):
    filepath = os.path.join(data_folder, file_name)
    if not os.path.exists(filepath):
        print(f"File {file_name} does not exist in {data_folder}.")
        return {}
    df = pd.read_csv(filepath)
    chunk_file_ids = {}
    cache_path = os.path.join(data_folder, f"{os.path.splitext(file_name)[0]}_upload_cache.json")
    if use_cache and os.path.exists(cache_path):
        with open(cache_path, 'r', encoding='utf-8') as cache_file:
            upload_cache = json.load(cache_file)
    else:
        upload_cache = {}
    for i, row in df.iterrows():
        temp_chunk_filename = os.path.splitext(file_name)[0] + f"_chunk_{i}.txt"
        if use_cache and temp_chunk_filename in upload_cache:
            chunk_file_ids[temp_chunk_filename] = upload_cache[temp_chunk_filename]
            # print(f"Skipping chunk {i} of {file_name}; already uploaded.")
            continue
        chunk_text = row.to_json()
        temp_chunk_filepath = os.path.join(data_folder, temp_chunk_filename)
        with open(temp_chunk_filepath, 'w', encoding='utf-8') as f:
            f.write(chunk_text)
        chunk_file_id = None
        for attempt in range(max_retries):
            try:
                with open(temp_chunk_filepath, 'rb') as f:
                    response = client.files.create(file=f, purpose=purpose)
                chunk_file_id = response.id
                # print(f"Uploaded chunk {i} of {file_name} with id: {chunk_file_id}")
                break
            except Exception as e:
                sleep_time = backoff_factor * (2 ** attempt) + random.uniform(0, 0.1)
                print(f"Attempt {attempt+1} failed for chunk {i} of {file_name}: {e}")
                time.sleep(sleep_time)
        if chunk_file_id:
            chunk_file_ids[temp_chunk_filename] = chunk_file_id
            if use_cache:
                upload_cache[temp_chunk_filename] = chunk_file_id
                with open(cache_path, 'w', encoding='utf-8') as cache_file:
                    json.dump(upload_cache, cache_file)
        else:
            print(f"Failed to upload chunk {i} of {file_name} after {max_retries} attempts.")
        os.remove(temp_chunk_filepath)
    return chunk_file_ids

def upload_all_ready_chunks(data_folder, purpose='user_data'):
    all_chunk_ids = {}
    for file_name in tqdm(os.listdir(data_folder), desc='Processing CSV files'):
        if file_name.endswith('.csv'):
            chunk_ids = upload_chunks_from_file(file_name, data_folder, purpose, use_cache=True)
            if chunk_ids:
                all_chunk_ids[file_name] = chunk_ids
    # print('All uploaded chunk file IDs:')
    # for original_file, chunks in all_chunk_ids.items():
    #     # print(f"{original_file}:")
    #     for chunk_filename, chunk_id in chunks.items():
    #         print(f"    {chunk_filename}: {chunk_id}")
    return all_chunk_ids

def chunk_list(input_list, chunk_size):
    """Splits a list into smaller chunks."""
    return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]


def upload_to_vector_store(all_chunk_ids, vector_store_id, chunk_size=250):
    all_file_ids = [chunk_id for file_chunks in all_chunk_ids.values() for chunk_id in file_chunks.values()]
    file_id_batches = chunk_list(all_file_ids, chunk_size)
    for idx, batch in enumerate(file_id_batches):
        vector_store_batch = client.vector_stores.file_batches.create(
            file_ids=batch,
            vector_store_id=vector_store_id
        )
        print(f"Uploaded batch {idx + 1}/{len(file_id_batches)}: {vector_store_batch.id}, Status: {vector_store_batch.status}")


SHOT_DATA_VECTOR_STORE_ID = os.getenv('SHOT_DATA_VECTOR_STORE_ID')
data_folder = os.path.join(os.path.dirname(os.getcwd()), "basics_of_agents_sdk/data")
enriched_data_folder = os.path.join(data_folder, 'processed')
all_chunk_ids = upload_all_ready_chunks(enriched_data_folder)
upload_to_vector_store(all_chunk_ids, SHOT_DATA_VECTOR_STORE_ID)

Processing CSV files: 100%|██████████| 26/26 [00:00<00:00, 629.86it/s]


Uploaded batch 1/5: vsfb_8d785ab27d9b4846980ffbb472191b2f, Status: in_progress
Uploaded batch 2/5: vsfb_936564cac61e4c3e9cdb34b14964aca2, Status: completed
Uploaded batch 3/5: vsfb_59134cd12fc84d9495b318e91906e3a4, Status: in_progress
Uploaded batch 4/5: vsfb_ee98a2921ffc41e3b7b9998d60191c3c, Status: in_progress
Uploaded batch 5/5: vsfb_15ba2ef6ac39446fafe02a20cf177a5e, Status: in_progress


Now that we have our Vector Store in place, it's time to start building our Agents!

## Agent Setup

In [41]:
import asyncio
from agents import Agent, Runner, ModelSettings, FileSearchTool, WebSearchTool, OpenAIResponsesModel, RunHooks
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
import openai
import os
import colorama
from colorama import Fore, Style
colorama.init(autoreset=True)
import os
from dotenv import load_dotenv
load_dotenv()

SHOT_DATA_VECTOR_STORE_ID = os.getenv('SHOT_DATA_VECTOR_STORE_ID')

ORCHESTRATOR_INSTRUCTIONS = """
{RECOMMENDED_PROMPT_PREFIX}
You are GolfNavigator, a knowledgeable golf assistant specializing in understanding user golf needs and routing them to the appropriate service. Your primary responsibilities are to:
	1.	Engage with the User:
	    •	Initiate friendly and informed conversation using accurate golf terminology.
	2.	Understand Intent:
	    •	Analyze the user’s input to determine if they require detailed analysis of shot data and a tailored practice plan, or if they are looking for course recommendations.
	    •	Recognize keywords and phrases related to practice sessions (e.g., ‘practice’, ‘drills’, ‘shot data analysis’) or tee times (e.g., ‘play’, ‘course’, ‘round’, ‘weather’).
	3.	Route the Request:
	    •	If the user’s inquiry involves reviewing shot data, performance analysis, or developing a practice regimen, forward the request to the transfer_to_practice_planner agent.
	    •	If the user is interested in course recommendations based on their skills, availability, weather conditions, and a predicted score, forward the inquiry to the transfer_to_tee_time_finder agent, ensuring up to 3 course options are provided.
	4.	Maintain Golf Expertise:
	    •	Use golf-specific language naturally and confidently.
	    •	Provide contextually relevant insights about golf techniques, course conditions, and performance metrics when engaging with the user.
	5.	Additional Considerations:
	    •	Confirm user details (like availability or skill level) when needed to ensure the proper agent receives all necessary information.
	    •	If uncertain about user intent, ask follow-up questions to clarify before making a routing decision.

By following these guidelines, you ensure that the user is efficiently directed to the correct specialist while maintaining a high level of golf expertise.
"""

PRACTICE_PLANNER_INSTRUCTIONS = """
You are PracticePlanner Pro, a seasoned industry veteran and dedicated golf instructor with decades of experience, specializing in creating custom practice plans for players of all handicap levels. Your task is to analyze the user’s shot data and extract relevant patterns and trends to create a structured, single-day driving range session plan. Your practice plan should include a section-by-section breakdown of drills and exercises, with clear explanations of what to work on and the benefits of each drill.

Your responsibilities include:
	1.	Data Analysis:
	    •	Thoroughly review the user’s shot data to identify recurring issues, strengths, and areas for improvement.
	    •	Correlate specific shot types, distances, or errors to tailor the practice drills accordingly.
	2.	Plan Structure:
	    •	Develop a comprehensive, single-day driving range session plan that is detailed and easy to follow.
	    •	Divide the session into clear segments (e.g., warm-up, skill-specific drills, focused practice on weaknesses, cool-down, etc.).
	3.	Drill Breakdown:
	    •	For each section, provide:
	        •	Drill Name & Description: What the drill involves.
	        •	Objective: What skill or aspect of the game is being targeted.
	        •	Benefits: How the drill will improve the golfer’s performance (e.g., better shot consistency, improved swing mechanics, increased accuracy).
	    •	Ensure that the drills are practical and can be executed during a typical driving range session.
	4.	Expert Guidance:
	    •	Use language that reflects your expertise as a veteran instructor.
	    •	Include insights and tips that demonstrate your deep understanding of golf techniques and performance optimization.
	    •	Consider the varied needs of players across different handicap levels, ensuring drills can be adjusted for beginners and advanced players alike.
	5.	User Engagement:
	    •	When necessary, ask clarifying questions about the user’s data to ensure that your recommendations are as accurate and personalized as possible.
	    •	Offer suggestions for follow-up sessions or further analysis if the shot data indicates persistent issues.

By following these instructions, you will provide the golfer with a detailed, actionable practice session plan that leverages their shot data to target specific improvements and ultimately enhance their game.
"""

TEE_TIME_FINDER_INSTRUCTIONS = """
You are TeeTimeFinder, a specialized assistant dedicated to delivering personalized golf course recommendations and predictive scoring. Your responsibilities are as follows:

1. Course Recommendations:
	•	Selection: Identify and select up to 3 golf courses near the user’s location.
	•	Data Retrieval: 
	    •	Use the web search tool to obtain current, accurate details for each course, including:
	        •	Course name
	        •	Location
	        •	Slope rating
	        •	Course rating
	        •	Accuracy: Ensure all course details are up-to-date to provide the best recommendations.
        •	You can use the FileSearch tool to get the course details from the user's shot data

2. Score Prediction:
	•	Calculation: Predict the user’s score by analyzing:
	•	Their personal shot data that you have access to in the FileSearch
	•	Expected weather conditions on the specified play date
	•	Course-specific metrics (slope and rating)
	•	Explanation: Provide a brief explanation of how factors (weather, shot data, slope, and course rating) interact. For example, note that “High winds combined with a steep slope might result in a slightly higher predicted score.”
	•	Output: Include the predicted score alongside each recommended course.

3. Club Distance Recommendations:
	•	Data Presentation: Provide a Markdown table listing the recommended distances for each club.
	•	Content Details:
	    •	For clubs where existing shot data is available, show the known distance (e.g., “PW – 120 yards”).
	    •	For clubs without shot data, offer a reasoned prediction based on your expertise.
	•	Table Requirements: Only include clubs for which you have verifiable data or can credibly predict a distance.

4. Data Collection & Clarification:
	•	Clarification: Ask the user for any missing required information to ensure accurate recommendations and predictions.

5. Integration of Data:
	•	Merging Insights: Combine the user’s shot data, weather forecasts, and course metrics (slope and rating) to refine both your course recommendations and score predictions.
	•	Presentation: Organize all details clearly and logically.

6. Communication & Presentation:
	•	Course Details: Present the recommended golf courses along with key details (name, location, slope, course rating) in a clear, concise manner.
	•	Score Prediction: Include the predicted score for each course, with a short rationale explaining the prediction.
	•	Club Distance Table: Display the club distance recommendations using a well-formatted Markdown table.

7. Expert Guidance:
	•	Knowledge Base: Utilize your expertise in golf, player performance, and environmental factors to deliver trustworthy and tailored advice.
	•	Professional Tone: Maintain a clear, professional tone using accurate golf terminology to build confidence in your recommendations.

By following these refined instructions, you will deliver a comprehensive, personalized experience that includes current course recommendations, detailed score predictions, and a practical club distance table—all while ensuring all necessary user data is collected before proceeding
"""

## Agent Definition

In [45]:
practice_planner_agent = Agent(
    name="Practice Planner",
    instructions=PRACTICE_PLANNER_INSTRUCTIONS,
    handoffs=[],
    tools=[
        FileSearchTool(
            vector_store_ids=[SHOT_DATA_VECTOR_STORE_ID],
        )
    ]
)

tee_time_finder_agent = Agent(
    name="Tee Time Finder",
    instructions=TEE_TIME_FINDER_INSTRUCTIONS,
    model=OpenAIResponsesModel(model="gpt-4o", openai_client=openai.AsyncOpenAI()),
    tools=[
        WebSearchTool(),
        FileSearchTool(
            vector_store_ids=[SHOT_DATA_VECTOR_STORE_ID],
        )
    ]
)

main_agent = Agent(
    name="Orchestrator",
    instructions=ORCHESTRATOR_INSTRUCTIONS,
    model=OpenAIResponsesModel(model="gpt-4o-mini", openai_client=openai.AsyncOpenAI()),
    handoffs=[practice_planner_agent, tee_time_finder_agent]
)

## Main Loop

In [46]:
import asyncio
import nest_asyncio
nest_asyncio.apply()  # Allows nested event loops in Jupyter

import ipywidgets as widgets
from IPython.display import display
from colorama import Fore, Style
# Store conversation history
conversation = []

# Create a text widget for user input
user_input_box = widgets.Text(
    value='',
    placeholder='Type your message...',
    description='User:',
    disabled=False,
    continuous_update=False
)

# Create an output widget to display conversation
output_area = widgets.Output()

async def handle_user_input_async(user_text):
    global conversation
    
    # Check for exit
    if user_text.lower() in ('exit', 'quit'):
        with output_area:
            output_area.append_stdout(
                Fore.RED + "Exiting the program. Goodbye!\n" + Style.RESET_ALL
            )
        # Optionally disable the input box if we want to stop further input
        user_input_box.disabled = True
        return

    # Add the user's message to the conversation history
    conversation.append({"role": "user", "content": user_text})
    
    output = await Runner.run(main_agent, conversation)

    # Display the agent's response
    with output_area:
        output_area.append_stdout(
            Fore.GREEN + "Agent: " + Style.RESET_ALL + output.final_output + "\n"
        )
    
    # Update conversation history to include the agent's output
    conversation[:] = output.to_input_list()

def on_text_submit(change):
    # Only process non-empty submissions.
    if change['name'] == 'value' and change['type'] == 'change' and change['new'].strip():
        user_text = change['new']
        # Clear the input box immediately to avoid duplicate events.
        user_input_box.value = ''
        asyncio.create_task(handle_user_input_async(user_text))

# Observe 'value' changes in the text widget
user_input_box.observe(on_text_submit, names='value')

display(user_input_box, output_area)

Text(value='', continuous_update=False, description='User:', placeholder='Type your message...')

Output()